In [ ]:
import grpc
import swoq_pb2
import swoq_pb2_grpc
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from time import sleep

In [ ]:
player_id = '6616b1c5bd0a697480a68319'

In [ ]:
UNKNOWN = 0
EMPTY = 1
PLAYER = 2
WALL = 3
EXIT = 4
DOOR_RED = 5
KEY_RED = 6
DOOR_GREEN = 7
KEY_GREEN = 8
DOOR_BLUE = 9
KEY_BLUE = 10

colors = {
    UNKNOWN:    [  0,   0,   0],
    EMPTY:      [ 50,  50,  50],
    PLAYER:     [  0,   0, 255],
    WALL:       [147, 124,  93],
    EXIT:       [230, 217, 177],
    DOOR_RED:   [128,   0,   0],
    KEY_RED:    [255,   0,   0],
    DOOR_GREEN: [  0, 128,  0],
    KEY_GREEN:  [  0, 255,   0],
    DOOR_BLUE:  [  0,   0, 128],
    KEY_BLUE:   [  0,   0, 255],
}

In [ ]:
INVENTORY_NONE = 0
INVENTORY_KEY_RED = 1
INVENTORY_KEY_GREEN = 2
INVENTORY_KEY_BLUE = 3

In [ ]:
def get_map_image(map, height, width):
    map_img = np.zeros((height, width, 3), dtype=np.float32)
    for y in range(height):
        for x in range(width):
            map_img[y, x] = (np.array(colors[map[y, x]]) / 255).astype(np.float32)
    return map_img

def plot_map(map, height, width):
    map_img = get_map_image(map, height, width)
    fig = plt.figure()
    plt.gca().set_axis_off()
    plt.tight_layout(pad=0)
    img = plt.imshow(map_img)
    plt.show()
    return (fig, img)

def update_map(frame, map, height, width):
    map_img = get_map_image(map, height, width)

    fig, img = frame
    img.set_data(map_img)
    clear_output(wait=True)
    display(fig)

In [ ]:
def update_global_state(state):
    global map, width, height, visibility_range
    global player_pos, inventory, finished

    player_pos = (state.playerPos.y, state.playerPos.x)
    inventory = state.inventory
    finished = state.finished
    print(f'{player_pos=}, {inventory=}, {finished=}')

    top = player_pos[0] - visibility_range
    left = player_pos[1] - visibility_range
    i = 0
    for y in range(visibility_range*2 + 1):
        for x in range(visibility_range*2 + 1):
            s = state.surroundings[i]
            if s != UNKNOWN:
                map_x = left + x
                map_y = top + y
                if 0 <= map_x < width and 0 <= map_y < height:
                    map[map_y, map_x] = s
            i += 1

In [ ]:
def start():
    global map, game_id, height, width, visibility_range
    
    with grpc.insecure_channel('localhost:5009') as channel:
        stub = swoq_pb2_grpc.TrainingStub(channel)
        startResponse = stub.StartGame(swoq_pb2.StartRequest(playerId=player_id, level=0))

    print(f'{startResponse.result=}')

    game_id = startResponse.gameId
    height = startResponse.height
    width = startResponse.width
    visibility_range = startResponse.visibilityRange
    print(f'{game_id=},{height=}, {width=}, {visibility_range=}')

    map = np.zeros((height, width), dtype=np.int8)

    update_global_state(startResponse.state)

start()
plot_map(map, height, width)

In [ ]:
def softmax(a):
    e = np.exp(a)
    return e / np.sum(e)

def pick_random_move(map):
    player_pos = np.unravel_index(np.argwhere(map == 2).flatten()[0], (height, width))
    target_pos = np.array([height-2, width-1])
    print(f'{player_pos=}, {target_pos=}')

    diff = target_pos - player_pos
    direction = np.sign(diff)

    # increase probability for actions in right direction
    moves = np.ones((4,))
    if direction[0] < 0: moves[0] += 1 # NORTH
    if direction[0] > 0: moves[1] += 1 # SOUTH
    if direction[1] < 0: moves[2] += 1 # WEST
    if direction[1] > 0: moves[3] += 1 # EAST
    print(f'{moves=}')
    moves = softmax(moves)

    # choose random move
    directions = [swoq_pb2.NORTH, swoq_pb2.SOUTH, swoq_pb2.WEST, swoq_pb2.EAST]
    direction = np.random.choice(directions, p=moves)
    print(f'{direction=}')
    return direction

In [ ]:
def move(direction):
    global frame
    with grpc.insecure_channel('localhost:5009') as channel:
        stub = swoq_pb2_grpc.TrainingStub(channel)
        moveResponse = stub.Move(swoq_pb2.ActionRequest(gameId=game_id, direction=direction))

    if moveResponse.result == 0:
        update_global_state(moveResponse.state)

    update_map(frame, map, height, width)
    print(f'{moveResponse.result=}')
    print(f'{moveResponse.state.finished=}')
    print(f'{moveResponse.state.inventory=}')

def use(direction):
    global frame
    with grpc.insecure_channel('localhost:5009') as channel:
        stub = swoq_pb2_grpc.TrainingStub(channel)
        useResponse = stub.Use(swoq_pb2.ActionRequest(gameId=game_id, direction=direction))

    if useResponse.result == 0:
        update_global_state(useResponse.state)
        
    update_map(frame, map, height, width)
    print(f'{useResponse.result=}')
    print(f'{useResponse.state.finished=}')
    print(f'{useResponse.state.inventory=}')

In [ ]:
def compute_distances():
    global map, player_pos
    
    todo = []
    distances = {}
    paths = {}

    todo.append(player_pos)
    distances[player_pos] = 0

    def is_solid(pos):
        global map
        cell = map[pos[0], pos[1]]
        return cell == WALL or cell == DOOR_RED or cell == DOOR_GREEN or cell == DOOR_BLUE or cell == UNKNOWN

    def enqueue(cur_pos, cur_dist, next_pos):
        nonlocal distances, todo, paths
        if not is_solid(next_pos):
            next_dist = distances[next_pos] if next_pos in distances else np.inf
            if cur_dist + 1 < next_dist:
                distances[next_pos] = cur_dist + 1
                todo.append(next_pos)
                paths[next_pos] = cur_pos

    while todo:
        cur_y, cur_x = cur_pos = todo[0]
        cur_dist = distances[cur_pos]
        del todo[0]
        
        if cur_y > 0: enqueue(cur_pos, cur_dist, (cur_y-1, cur_x))
        if cur_y < height-1: enqueue(cur_pos, cur_dist, (cur_y+1, cur_x))
        if cur_x > 0: enqueue(cur_pos, cur_dist, (cur_y, cur_x-1))
        if cur_x < width-1: enqueue(cur_pos, cur_dist, (cur_y, cur_x+1))
    
    return distances, paths

In [ ]:

def get_direction_towards(target_pos, paths):
    global map, player_pos
    
    if target_pos not in paths: return None
    
    # Get first step in path towards target
    cur = target_pos
    prev = None
    while cur != player_pos:
        prev = cur
        cur = paths[cur]
    next_pos = prev
    assert(next_pos is not None)
    
    diff_y = next_pos[0] - player_pos[0]
    diff_x = next_pos[1] - player_pos[1]
    if diff_y > 0: return swoq_pb2.SOUTH
    if diff_y < 0: return swoq_pb2.NORTH
    if diff_x > 0: return swoq_pb2.EAST
    if diff_x < 0: return swoq_pb2.WEST
    
    return None

In [ ]:
def get_direction_towards_closest_unknown():
    global map

    distances, paths = compute_distances()

    closest_empty = None
    closest_dist = None

    # find closest empty with UNKNOWN neighbors
    for pos in np.argwhere(map == EMPTY):
        pos = tuple(pos)
        pos_y, pos_x = pos
        if map[pos_y, pos_x-1] == UNKNOWN or \
                map[pos_y, pos_x+1] == UNKNOWN or \
                map[pos_y+1, pos_x] == UNKNOWN or \
                map[pos_y-1, pos_x] == UNKNOWN:
            dist = distances[pos]
            if closest_dist is None or dist < closest_dist:
                closest_dist = dist
                closest_empty = pos
               
    if closest_empty is None:
        return None

    return get_direction_towards(closest_empty, paths)

In [ ]:
def explore():
    while True:
        direction = get_direction_towards_closest_unknown()
        if direction is None: break
        move(direction)
        #sleep(0.05)

frame = plot_map(map, height, width)
explore()

In [ ]:
def try_reach_exit():
    global map
    
    exit_pos = np.argwhere(map == EXIT)
    if not np.any(exit_pos): return
    
    exit_pos = exit_pos[0]
    while True:
        _, paths = compute_distances()
        direction = get_direction_towards(tuple(exit_pos), paths)
        if direction is None: break
        move(direction)

frame = plot_map(map, height, width)
try_reach_exit()

In [ ]:
def try_get_key():
    global map, inventory
    while inventory == INVENTORY_NONE:
        distances, paths = compute_distances()
        
        doors = np.argwhere(map == DOOR_RED)
        keys = np.argwhere(map == KEY_RED)
        if np.any(doors) and np.any(keys):
            direction = get_direction_towards(tuple(keys[0]), paths)
            assert(direction is not None)
            move(direction)
            continue

        doors = np.argwhere(map == DOOR_GREEN)
        keys = np.argwhere(map == KEY_GREEN)
        if np.any(doors) and np.any(keys):
            direction = get_direction_towards(tuple(keys[0]), paths)
            assert(direction is not None)
            move(direction)
            continue

        doors = np.argwhere(map == DOOR_BLUE)
        keys = np.argwhere(map == KEY_BLUE)
        if np.any(doors) and np.any(keys):
            direction = get_direction_towards(tuple(keys[0]), paths)
            assert(direction is not None)
            move(direction)
            continue

frame = plot_map(map, height, width)
try_get_key()

In [ ]:
def try_open_door():
    global map, inventory
    
    target_pos = None
    use_direction = None
    
    def find_target(door):
        nonlocal target_pos, use_direction
        global map
        
        top = (door[0]-1, door[1])
        if target_pos is None and map[top[0], top[1]] == EMPTY:
            target_pos = top
            use_direction = swoq_pb2.SOUTH
        bottom = (door[0]+1, door[1])
        if target_pos is None and map[bottom[0], bottom[1]] == EMPTY:
            target_pos = bottom
            use_direction = swoq_pb2.NORTH
        left = (door[0], door[1]-1)
        if target_pos is None and map[left[0], left[1]] == EMPTY:
            target_pos = left
            use_direction = swoq_pb2.EAST
        right = (door[0], door[1]+1)
        if target_pos is None and map[right[0], right[1]] == EMPTY:
            target_pos = right
            use_direction = swoq_pb2.WEST
    
    if target_pos is None and inventory == INVENTORY_KEY_RED:
        doors = np.argwhere(map == DOOR_RED)
        if np.any(doors): find_target(doors[0])

    if target_pos is None and inventory == INVENTORY_KEY_GREEN:
        doors = np.argwhere(map == DOOR_GREEN)
        if np.any(doors): find_target(doors[0])

    if target_pos is None and inventory == INVENTORY_KEY_BLUE:
        doors = np.argwhere(map == DOOR_BLUE)
        if np.any(doors): find_target(doors[0])
                
    if target_pos is not None:
        while True:
            _, paths = compute_distances()
            direction = get_direction_towards(target_pos, paths)
            if direction is None: break
            move(direction)
        use(use_direction)
            
frame = plot_map(map, height, width)
try_open_door()

In [ ]:
frame = plot_map(map, height, width)
explore()

In [ ]:
start()
frame = plot_map(map, height, width)

while not finished:
    explore()
    try_reach_exit()
    if finished: break
    try_get_key()
    try_open_door()